In [1]:
import pandas as pd
import numpy as np

In [2]:
from datetime import date

today = date.today() # date object
# Format
date_formatted = today.strftime("%m-%d-%Y")

In [3]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '1PLrnOq_Dyc2pBCbNUz1eUIh4YwAQlW1C3nOe99k5klY' +
                   '/export?gid=0&format=csv')
#df.head()

In [4]:
df.drop(['Attributes Ph Masterl','Attributes Fid', 'Attributes Travel Hx', 'Attributes Symptoms',
          'Attributes Latitude', 'Attributes Longitude', 'Geometry X',
          'Geometry Y', 'Attributes Epi Link', 'Attributes Petsa'], axis=1, inplace=True) 
df.columns = ['case_no', 'age', 'sex', 'nationality', 'residence', 'date_confirmed', 'hospital', 'status']
df.set_index(df.case_no,inplace=True)
df.sort_index(inplace=True) # for some reason this crates a redundant case_no column
df.drop(['case_no'], axis=1, inplace=True)


with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df)

,age,sex,nationality,residence,date_confirmed,hospital,status
case_no,,,,,,,
1,38,Female,Chinese,None,1/30/2020,SAN LAZARO HOSPITAL,
2,44,Male,Chinese,None,1/30/2020,SAN LAZARO HOSPITAL,
3,60,Female,Chinese,None,1/30/2020,For validation,
4,48,Male,Filipino,Taguig City,3/5/2020,U.E.R.M. MEMORIAL HOSPITAL,
5,62,Male,Filipino,"Cainta, Rizal",3/7/2020,RESEARCH INSTITUTE FOR TROPICAL MEDICINE,
6,59,Female,Filipino,"Cainta, Rizal",3/7/2020,RESEARCH INSTITUTE FOR TROPICAL MEDICINE,
7,38,Male,"Chinese, Taiwanese",Makati City,3/8/2020,RESEARCH INSTITUTE FOR TROPICAL MEDICINE,
8,32,Male,Filipino,Pasig City,3/8/2020,ST. LUKE'S MEDICAL CENTER-GLOBAL CITY,
9,86,Male,American,Marikina City,3/8/2020,THE MEDICAL CITY,


# Cleaning

In [ ]:
# Replacing the enyes

In [5]:
txt = 'City of Para�aque, NCR, Fourth District'
'�' in txt


True

In [ ]:
def enye(name):
    if '�' in name:
        return 
    else:
        return name

In [ ]:
df.hospital

# Hospital locations using Folium

In [ ]:
hosp_df = df.hospital.value_counts().rename_axis('hospital').to_frame('count')
hosp_df

### Geocoding

# Age analysis

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()

plt.rcParams.update({'font.size': 10})
plt.style.use('seaborn-dark')

ax = plt.subplot(111)

bins = np.array([0, 10, 20, 30, 40, 50,60,70,80,90,100])
n, bins, patches = ax.hist(df.age, bins, histtype='bar', color='#936ED4',rwidth=0.8, align='left') #, color='#a500ff'
patches[np.argmax(n)].set_fc('#FCB249') #color the highest


# label each bar with each count
for index, bin_ in enumerate(bins):
    if bin_== 100:
        break
        
    ax.text(bin_, n[index] + 10, int(n[index]), 
            color='black', fontweight='bold', 
            horizontalalignment='center', verticalalignment='center')

#Styling
ax.set_title('Number of Infected Per Age Group')
    
# xtick labels
plt.xticks(bins)
ax.set_xlim(-10,99.9)
ax.set_ylim(0,np.max(n)+50)
    
# Removing spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)

# Removing yticks
ax.get_yaxis().set_visible(False)

# Removing ticks
ax.tick_params(axis=u'both', which=u'both',length=0)

# Draw the plot first then adjust the ticklabels
fig.canvas.draw()
labels = np.array([item.get_text() for item in ax.get_xticklabels()])
new_labels = []
for i in range(0,len(labels)):
    new_labels.append(labels[i] + 's')
ax.set_xticklabels(new_labels)

# Time stamp
ax.text(80, -60, 'As of {}'.format(date_formatted), fontsize=8)

In [ ]:
plt.savefig('images/num_infected_{}'.format(date_formatted),  dpi=500, bbox_inches='tight')

In [ ]:
# Sanity check

sum([60 <= i <= 69 for i in df.age])